## Script based on base_mutator.py written by Dennis Krüger

Krüger, Dennis M.; Kamerlin, Shina Caroline Lynn (2018): DNA Base Mutator. Zenodo. Software. https://doi.org/10.5281/zenodo.1494296 

This notebook canbe used with a pdb file containing a DNA and a sequence file, text file, that contains a line of bases. The number of bases in the sequence file must match the number of bases in the pdb file. This is for test purposes and was used when prepping the starting structure for 1efa structure containing the 21 bp operator O1.

In [1]:
import os, sys, math, random, argparse, time, shutil
import pymol
from pymol import cmd
import numpy as np

In [2]:

"""
This is important as soon as you convert the notebook into a little python program

class MyParser(argparse.ArgumentParser):
    def error(self, message):
        sys.stderr.write('error: %s\n' % message)
        self.print_help()
        sys.exit(2)

def check_file(x):
    if os.path.isfile(x) == False:
        raise argparse.ArgumentTypeError("File does not exist")
    return(x)

parser=MyParser(description='Syntax description')
parser.add_argument('-p', type=check_file, help='PDB file ', required=True)
parser.add_argument('-s', type=check_file, help='Sequence file ', required=True)
"""

'\nThis is important as soon as you convert the notebook into a little python program\n\nclass MyParser(argparse.ArgumentParser):\n    def error(self, message):\n        sys.stderr.write(\'error: %s\n\' % message)\n        self.print_help()\n        sys.exit(2)\n\ndef check_file(x):\n    if os.path.isfile(x) == False:\n        raise argparse.ArgumentTypeError("File does not exist")\n    return(x)\n\nparser=MyParser(description=\'Syntax description\')\nparser.add_argument(\'-p\', type=check_file, help=\'PDB file \', required=True)\nparser.add_argument(\'-s\', type=check_file, help=\'Sequence file \', required=True)\n'

In [13]:
"Read in the files.  "
sFileS='sequence_O1.txt'
sFileP='1efa_O1_chains_DG5_moved_mod.pdb'

In [4]:
#try:
#args = parser.parse_args()
#except:
#    print("\n")
#    sys.exit()
#    cmd.quit()

#sFileS = args.s
#sFileP = args.p
sPath = os.getcwd()

sBaseData_DA = [
'ATOM      1  C1\'  DA     1      -1.898 118.540  -7.924  1.00  0.00           C',
'ATOM      2  N1   DA     1      -3.988 117.321  -3.422  1.00  0.00           N',
'ATOM      3  C2   DA     1      -2.735 117.271  -3.888  1.00  0.00           C',
'ATOM      4  N3   DA     1      -2.270 117.599  -5.089  1.00  0.00           N',
'ATOM      5  C4   DA     1      -3.266 118.029  -5.883  1.00  0.00           C',
'ATOM      6  C5   DA     1      -4.605 118.137  -5.553  1.00  0.00           C',
'ATOM      7  C6   DA     1      -4.969 117.759  -4.241  1.00  0.00           C',
'ATOM      8  N6   DA     1      -6.216 117.809  -3.762  1.00  0.00           N',
'ATOM      9  N7   DA     1      -5.336 118.622  -6.631  1.00  0.00           N',
'ATOM     10  C8   DA     1      -4.435 118.795  -7.572  1.00  0.00           C',
'ATOM     11  N9   DA     1      -3.159 118.463  -7.183  1.00  0.00           N']

sBaseData_DG = [
'ATOM      1  C1\'  DG     1      14.428  -7.039  -9.545  1.00  0.00           C',
'ATOM      2  N1   DG     1      10.104  -5.997 -12.002  1.00  0.00           N',
'ATOM      3  C2   DG     1      11.061  -5.135 -11.514  1.00  0.00           C',
'ATOM      4  N2   DG     1      10.857  -3.825 -11.736  1.00  0.00           N',
'ATOM      5  N3   DG     1      12.142  -5.527 -10.859  1.00  0.00           N',
'ATOM      6  C4   DG     1      12.192  -6.868 -10.721  1.00  0.00           C',
'ATOM      7  C5   DG     1      11.290  -7.816 -11.166  1.00  0.00           C',
'ATOM      8  C6   DG     1      10.135  -7.382 -11.877  1.00  0.00           C',
'ATOM      9  O6   DG     1       9.222  -8.070 -12.367  1.00  0.00           O',
'ATOM     10  N7   DG     1      11.704  -9.093 -10.816  1.00  0.00           N',
'ATOM     11  C8   DG     1      12.832  -8.905 -10.182  1.00  0.00           C',
'ATOM     12  N9   DG     1      13.196  -7.581 -10.110  1.00  0.00           N']

sBaseData_DT = [
'ATOM      1  C1\'  DT     1      -5.939 126.836  -5.838  1.00  0.00           C',
'ATOM      2  N1   DT     1      -5.790 125.997  -4.632  1.00  0.00           N',
'ATOM      3  C2   DT     1      -4.624 125.275  -4.511  1.00  0.00           C',
'ATOM      4  O2   DT     1      -3.752 125.272  -5.368  1.00  0.00           O',
'ATOM      5  N3   DT     1      -4.513 124.558  -3.345  1.00  0.00           N',
'ATOM      6  C4   DT     1      -5.428 124.490  -2.311  1.00  0.00           C',
'ATOM      7  O4   DT     1      -5.174 123.815  -1.312  1.00  0.00           O',
'ATOM      8  C5   DT     1      -6.640 125.255  -2.513  1.00  0.00           C',
'ATOM      9  C6   DT     1      -6.761 125.956  -3.651  1.00  0.00           C',
'ATOM     10  C7   DT     1      -7.695 125.235  -1.453  1.00  0.00           C']

sBaseData_DC = [
'ATOM      1  C1\'  DC     1      -1.323 109.699  -3.137  1.0  0.00           C',
'ATOM      2  N1   DC     1      -2.591 109.952  -3.837  1.00  0.00           N',
'ATOM      3  C2   DC     1      -3.612 110.589  -3.137  1.00  0.00           C',
'ATOM      4  O2   DC     1      -3.423 110.887  -1.947  1.00  0.00           O',
'ATOM      5  N3   DC     1      -4.775 110.869  -3.769  1.00  0.00           N',
'ATOM      6  C4   DC     1      -4.941 110.527  -5.046  1.00  0.00           C',
'ATOM      7  N4   DC     1      -6.106 110.840  -5.624  1.00  0.00           N',
'ATOM      8  C5   DC     1      -3.922 109.855  -5.784  1.00  0.00           C',
'ATOM      9  C6   DC     1      -2.772 109.589  -5.145  1.00  0.00           C']

In [5]:
fDA = open("DA.pdb","w")
for i in range(len(sBaseData_DA)):
    fDA.write(sBaseData_DA[i]+"\n")
fDA.close()

fDG = open("DG.pdb","w")
for i in range(len(sBaseData_DG)):
    fDG.write(sBaseData_DG[i]+"\n")
fDG.close()

fDT = open("DT.pdb","w")
for i in range(len(sBaseData_DT)):
    fDT.write(sBaseData_DT[i]+"\n")
fDT.close()

fDC = open("DC.pdb","w")
for i in range(len(sBaseData_DC)):
    fDC.write(sBaseData_DC[i]+"\n")
fDC.close()


In [7]:
aAtoms = ["P","OP1","OP2","O3'","O5'","C1'","C2'","C3'","C4'","C5'","O3'","O4'","O5'","N1","C6","N9","C8"]

sSeqS = ""

#read the sequence file
f1 = open(sFileS,"r") 
sSeqS += f1.readline().strip() #reads the bases into sSegS1, gets rid of the string ending
f1.close()

In [8]:
sSeqS

'CCAATTGTGAGCGGATAACAATTCCGGAATTGTTATCCGCTCACAATTGG'

In [9]:
def DNAProteinFitBases(file):
    "These are just some of the DNA atoms, which will be used to fit the new nucleotides."
    fitAtoms = ["P","OP1","OP2","O3'","O5'","C1'","C2'","C3'","C4'","C5'","O3'","O4'","O5'","N1","C6","N9","C8"]
    f2 = open(file,"r")
    "Find the of the start and ends in the chain. This checks for CRYST header and termini, TER."
    lines=f2.readlines()
    Protein=[]
    DNA=[]
    DNAs=[]
    FitDNA=[]
    FitDNAs=[]
    IDs=[]
    Res=[]
    if ("CRYST") in lines[0]:
        Start=1
        "The number of termini give you the number of  molecules:"
        Index=[]
        for num, x in enumerate(lines, start=Start):
            if x[:3]=="TER":
                Index.append(num)
    else:
        Start=0
        "The number of termini give you the number of  molecules:"
        Index=[]
        for num, x in enumerate(lines, start=Start):
            if x[:3]=="TER":
                Index.append(num+1)
    print(Index)
    "Iterate over the molecules. You get the Protein lines with a final TER line."
    Start1=Start
    for TER_index in Index:
        End=TER_index
        #print(Start1,End)
        ProteinLines=[]
        DNAfit1=[]
        DNALines=[]
        DNAids=[]

        "Iterate over the lines in one molecule."
        molecule=[]
        i=Start1
        while i<End:
            molecule.append(lines[i])
            i+=1
        #print(molecule)
        if molecule[0][17:20].strip()[:2] in ["DA","DT","DG","DC"]:
            #print("DNA molecule")
            DNA.append(molecule)
            DNAids.append([str(x[23:26].strip()) for x in molecule])
            "Check for the atoms in the DNA that will be used for fitting of the new nucleotides."
            for line in molecule:
                if(line[13:17].strip() in fitAtoms):
                    if(line[13:17].strip() == "N9" or line[13:17].strip() == "C8"):
                        if(line[17:20].strip()[:2] == "DA" or line[17:20].strip()[:2] == "DG"):
                            DNAfit1.append(line)
                    if(line[13:17].strip() == "N1" or line[13:17].strip() == "C6"):
                        if(line[17:20].strip()[:2] == "DC" or line[17:20].strip()[:2] == "DT"):
                            DNAfit1.append(line)
                    if(line[13:17].strip() != "N9" and line[13:17].strip() != "C8" and line[13:17].strip() != "N1" and line[13:17].strip() != "C6"):
                        DNAfit1.append(line)
            FitDNA.append(DNAfit1)
            IDs.append(DNAids)
            
        else:
            #print("Protein molecule")
            Protein.append(molecule)     
        Start1=TER_index
    f2.close()
    "How many DNA molecules do you have?"
    MolN=len(IDs)
    "You want to have the DNA split into single bases."
    for i in range(MolN):
        Sequence=[]
        DNAsplit=[]
        DNALines=DNA[i]
        DNAResIDs=np.sort(list(set(IDs[i][0])))
        print(DNAResIDs)
        j=0
        while j < len(DNAResIDs)-1:
            #print(DNAResIDs[j+1])
            residueLines=[]
            for line in DNALines:
                #print(line[23:26],DNAResIDs[j])
                if line[23:26]==DNAResIDs[j+1]:
                    ResidueName=line[17:20].strip()
                    residueLines.append(line)
            #print(residueLines)
            DNAsplit.append(residueLines)
            Sequence.append(ResidueName)
            j+=1
        DNAs.append(DNAsplit) 
        Res.append(Sequence)
    for i in range(MolN):
        DNAsplit=[]
        DNALines=FitDNA[i]
        DNAResIDs=np.sort(list(set(IDs[i][0])))
        j=0
        while j < len(DNAResIDs)-1:
            residueLines=[]
            for line in DNALines:
                if line[23:26]==DNAResIDs[j+1]:
                    residueLines.append(line)
            DNAsplit.append(residueLines)
            j+=1
        FitDNAs.append(DNAsplit) 
    ResidueIDs=list(set(([x for x in IDs[0][0]+IDs[1][0] if x!=""])))
    return(DNAs,Protein,FitDNAs,ResidueIDs,Res)

In [14]:
DNA,Protein,FitBases,ids,res=DNAProteinFitBases(sFileP)

[2519, 5038, 5548, 6059]
['' '671' '672' '673' '674' '675' '676' '677' '678' '679' '680' '681'
 '682' '683' '684' '685' '686' '687' '688' '689' '690' '691' '692' '693'
 '694' '695']
['' '696' '697' '698' '699' '700' '701' '702' '703' '704' '705' '706'
 '707' '708' '709' '710' '711' '712' '713' '714' '715' '716' '717' '718'
 '719' '720']


In [15]:
"This checks if you have the right base identifiers and reshapes the the list from the sequence file."
for i in range(len(sSeqS)):
    if(sSeqS[i] not in ["A","T","G","C"]):
        print(" ERROR: Wrong base identifier. "+sSeqS[i])
        sys.exit()
        cmd.quit()
"Tests if the sequence in the pdb file and the new sequence in the sequence file have the same length."
if(len(ids) != len(sSeqS)):
    print(" ERROR: Sequences have different lengths. PDBfile:"+str(len(ids))+" Sequence file:"+str(len(sSeqS)))
    sys.exit()
    cmd.quit()

In [16]:
def mutateChain(DNAbB,seqPDB,NewSeq,DNA,Protein):
    """Iterate over the bases of the DNA bB plus two purine/pyrimidine residues, 
    that you will fit on and keep the backbone of.
    For your case this needs to be done per DNA molecule in FitBases. """
    "How many DNA molecules do you have?"
    "Open the final output file."
    fFinal = open(sFileP[:-4]+"_mod.pdb","w")
    Protein_all=[x for y in Protein for x in y]
    for line in Protein_all:
        fFinal.write(line) 
    MolN=len(seqPDB)
    #print(MolN)
    "Get the new sequence."
    aNewSeq=list(NewSeq)
    BaseSeqSplit=np.array_split(aNewSeq,MolN)
    #print(BaseSeqSplit)
    "Iterate over the DNA molecules."
    #Chains=["C","D","E","F"]
    "Get the first AtomID of the Bases to count from here and put all new AtomIDs."
    AtomID=int(DNA[0][0][0][7:11].strip())
    BaseID=int(DNA[0][0][0][23:26].strip()) #This needs to go to the mutated ones

    print(AtomID)
    for i in range(MolN):
        #Chain=Chains[i]
        PDBChain=seqPDB[i]
        Bases=DNAbB[i] #This is used in case 
        BasesFull=DNA[i]
        print(PDBChain)
        ChainID=Bases[0][0][21:22]
        SequenceChain=BaseSeqSplit[i]
        sBaseSeq = ["D"+i for i in SequenceChain]
        sBaseSeq[0]=sBaseSeq[0]+"5"
        sBaseSeq[-1]=sBaseSeq[-1]+"3"
        print(sBaseSeq)
        "Iterate over the bases."
        for j in range(len(sBaseSeq)):
            Base=Bases[j]
            BaseFull=BasesFull[j]
            BaseName=sBaseSeq[j]
            if sBaseSeq[j]!=PDBChain[j]:
                
                print('Mutate to {}!'.format(BaseName))
                "Write the fitBase for the mutation to a temporary file."
                fTmp = open("tmp_vec.pdb","w")
                for line in Base:
                    fTmp.write(line)
                fTmp.close()
                #print(Base)
                sFileName = sBaseSeq[j]+".pdb" #This defines which of the bases to fit
                print(sFileName)
                "PyMol settings"
                cmd.feedback("disable","all","actions")
                cmd.feedback("disable","all","results")
                cmd.load(sFileName,"base") #load the base to fit, give it the name "base"
                cmd.load("tmp_vec.pdb","mutated") #load the base that will be mutated, give it the name "mutated"

                "Selection for the base in the new sequence."
                if(sBaseSeq[j] == "DA" or sBaseSeq[j] == "DG"): 
                    sele1 = ["C1'","N9", "C8"]#for purines you will fit on C1´, N9 and C8
                    print(sele1)
                else:
                    sele1 = ["C1'","N1", "C6"]#for pyrimidines you will fit on C1', N1 and C6
                "Selection for the base in the PDB file."
                if(Base[0][17:20].strip() == "DA" or Base[0][17:20].strip() == "DG" or Base[0][17:20].strip() == "DG5"):
                    sele2 = ["C1'","N9", "C8"]
                    print(sele2)
                else:
                    sele2 = ["C1'","N1", "C6"]
                "Here comes the fitting."
                cmd.pair_fit("base///"+sBaseSeq[j]+"/"+sele1[0],"mutated///"+Base[0][17:20].strip()+"/"+sele2[0],"base///"+sBaseSeq[j]+"/"+sele1[1],"mutated///"+Base[0][17:20].strip()+"/"+sele2[1],"base///"+sBaseSeq[j]+"/"+sele1[2],"mutated///"+Base[0][17:20].strip()+"/"+sele2[2])
                "This saves the new base according to the sequence file."
                cmd.save("tmp_base.pdb","base")
                cmd.reinitialize()
                """"Now you have the fitted base in the tmp_base file."
                You can strip the fitting nucleotide residues."""
                for line in Base:
                    if (line[13:17].strip() != "N9" and line[13:17].strip() != "C8" and line[13:17].strip() != "N1" and line[13:17].strip() != "C6"):
                        
                        #line[7:11]=AtomID
                        temp = list(line)
                        temp[17:20] = "{:>3s}".format(BaseName)
                        temp[23:26] = "{:.0f}".format(BaseID)
                        temp[7:11] = str(AtomID)
                        line = "".join(temp)
                        fFinal.write(line)
                        AtomID+=1
                    BaseID+=1
                print(BaseID)
                fTmp = open("tmp_base.pdb","r")
                for line in fTmp.readlines()[:-2]:
                    if (line[13:17].strip() != "C1'"):
                        "Replace the Residue- and AtomIDs."
                        temp = list(line)
                        temp[17:20] = "{:>3s}".format(BaseName)
                        temp[23:26] = "{:.0f}".format(BaseID)
                        temp[21:22] = ChainID
                        temp[7:11] = "{:.0f}".format(AtomID)
                        line = "".join(temp)
                        #temp = list(line)
                        #
                        #line = "".join(temp)
                        fFinal.write(line)
                        AtomID+=1
                fTmp.close()
                BaseID+=1
            else:
                for line in BaseFull:
                    temp = list(line)
                    temp[7:11] = "{:.0f}".format(AtomID)
                    temp[17:20] = "{:>3s}".format(BaseName)
                    temp[23:26] = "{:.0f}".format(BaseID)
                    line = "".join(temp)
                    fFinal.write(line)
                    AtomID+=1
                BaseID+=1
        fFinal.write('TER\n')
    fFinal.write('END\n')
    fFinal.close()          
    return(Base)

In [17]:
bbtest=mutateChain(FitBases,res,sSeqS,DNA,Protein)

5037
['DC5', 'DC', 'DA', 'DA', 'DT', 'DT', 'DG', 'DT', 'DG', 'DA', 'DG', 'DC', 'DG', 'DG', 'DA', 'DT', 'DA', 'DA', 'DC', 'DA', 'DA', 'DT', 'DT', 'DC', 'DC3']
['DC5', 'DC', 'DA', 'DA', 'DT', 'DT', 'DG', 'DT', 'DG', 'DA', 'DG', 'DC', 'DG', 'DG', 'DA', 'DT', 'DA', 'DA', 'DC', 'DA', 'DA', 'DT', 'DT', 'DC', 'DC3']
['DG5', 'DG', 'DA', 'DA', 'DT', 'DT', 'DG', 'DT', 'DT', 'DA', 'DT', 'DC', 'DC', 'DG', 'DC', 'DT', 'DC', 'DA', 'DC', 'DA', 'DA', 'DT', 'DT', 'DG', 'DG3']
['DG5', 'DG', 'DA', 'DA', 'DT', 'DT', 'DG', 'DT', 'DT', 'DA', 'DT', 'DC', 'DC', 'DG', 'DC', 'DT', 'DC', 'DA', 'DC', 'DA', 'DA', 'DT', 'DT', 'DG', 'DG3']


In [18]:
bbtest

['ATOM   6034  P   DG3 D 720      13.102  35.656 -35.551  1.00  0.00           P  \n',
 "ATOM   6035  C5' DG3 D 720      10.685  35.485 -34.550  1.00  0.00           C  \n",
 "ATOM   6036  O5' DG3 D 720      11.510  35.635 -35.722  1.00  0.00           O  \n",
 "ATOM   6037  C4' DG3 D 720       9.518  36.452 -34.613  1.00  0.00           C  \n",
 "ATOM   6038  O4' DG3 D 720       9.843  37.751 -34.037  1.00  0.00           O  \n",
 "ATOM   6039  C3' DG3 D 720       9.006  36.788 -36.013  1.00  0.00           C  \n",
 "ATOM   6040  O3' DG3 D 720       7.592  36.921 -35.966  1.00  0.00           O  \n",
 "ATOM   6041  C2' DG3 D 720       9.614  38.153 -36.331  1.00  0.00           C  \n",
 "ATOM   6042  C1' DG3 D 720       9.423  38.758 -34.943  1.00  0.00           C  \n",
 'ATOM   6052  C8  DG3 D 720      11.517  40.271 -35.158  1.00  0.00           C  \n',
 'ATOM   6053  N9  DG3 D 720      10.252  39.972 -34.700  1.00  0.00           N  \n',
 'ATOM   6054  OP1 DG3 D 720      13.621  3